In [ ]:
import os
import cv2
import winsound
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score

In [ ]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False 
    results = model.process(image)  
    image.flags.writeable = False 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                            )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                            )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                            )

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
DATA_PATH = os.path.join('DATA') 
actions = [folder for folder in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, folder))]
no_sequences = 30
sequence_length = 30

In [ ]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
from PIL import Image,ImageFont, ImageDraw

cap = cv2.VideoCapture(0)
font_path = "SolaimanLipi/SolaimanLipi_22-02-2012.ttf"


def render_text(image, text, position, font_path, font_size, font_color):
    
    font = ImageFont.truetype(font_path, font_size)
    
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)
    
    draw.text(position, text, font=font, fill=font_color)
    
    return np.array(pil_image)


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   
    for action in actions:
        
        for sequence in range(no_sequences):
           
            for frame_num in range(sequence_length):

              
                ret, frame = cap.read()

               
                image, results = mediapipe_detection(frame, holistic)

               
                draw_styled_landmarks(image, results)
                
                
                if frame_num == 0: 
                 
                    image = render_text(image, 'STARTING COLLECTION', (120,200), font_path, 20, (0,255, 0))
                    image = render_text(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), font_path, 10, (0, 0, 255))
                   
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    
                    image = render_text(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), font_path, 10, (0, 0, 255))
                   
                    cv2.imshow('OpenCV Feed', image)
                
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

In [ ]:
res = [.7, 0.2, 0.1]

In [ ]:
actions[np.argmax(res)]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('499model.keras')

model.summary()

In [ ]:
res = model.predict(X_test)

In [ ]:
print("Preditecd:", actions[np.argmax(res[0])])
print("Actual:", actions[np.argmax(y_test[0])])

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
f1_score(ytrue, yhat, average="macro")

In [ ]:
model.save('499model.keras')

In [ ]:
from PIL import ImageFont, ImageDraw, Image

sequence = []
sentence = []
threshold = 0.4
cap = cv2.VideoCapture(0)
font_path = "SolaimanLipi/kalpurush.ttf"

def render_text(image, text, position, font_path, font_size, font_color):
    text = text.encode('utf-8').decode('utf-8')
    # Load the font
    font = ImageFont.truetype(font_path, font_size)
    # Convert the OpenCV image to a PIL image
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)
    # Render the text onto the PIL image
    draw.text(position, text, font=font, fill=font_color)
    # Convert the PIL image back to an OpenCV image
    return np.array(pil_image)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()  # Capture the video frame by frame
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Make detection
        draw_styled_landmarks(image, results)  # Draw
        
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            # Compare each element of the array with the threshold
            if np.any(res > threshold):
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

        if len(sentence) > 3:
            sentence = sentence[-3:]

        # Render the text on the frame
        text_to_display = '   '.join(sentence).encode('utf-8').decode('utf-8')
        image_with_text = render_text(image, text_to_display, (11, 50), font_path, 1, (255, 255, 255))

        # Create a black background box for displaying the text outside the frame
        text_box = np.zeros((100, image_with_text.shape[1], 3), dtype=np.uint8)
        text_box[:] = (0, 0, 0)  # Black background
        text_box = render_text(text_box, text_to_display, (10, 10), font_path, 24, (255, 255, 255))

        # Combine the image and the text box
        combined_image = np.vstack((image_with_text, text_box))

        cv2.imshow('Real time Predict', combined_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()